Import dependance and load csv files

In [13]:
from cv2 import mean
import pandas as pd
import matplotlib.pyplot as plt
import datetime

stops = pd.read_csv('data_sncf/stops.txt')
stops_times = pd.read_csv('data_sncf/stop_times.txt')
routes = pd.read_csv('data_sncf/routes.txt')
trips = pd.read_csv('data_sncf/trips.txt')
calendar_dates = pd.read_csv('data_sncf/calendar_dates.txt')
calendar = pd.read_csv('data_sncf/calendar.txt')
agency = pd.read_csv('data_sncf/agency.txt')



# Link stop id and stop names

Remove duplicate stop names :

In [14]:
ustops=stops.dropna(subset=['stop_id','stop_name'])
ustops=ustops.drop_duplicates(subset=['stop_name'])
#df=pd.merge(left=stops[['stop_id','stop_name']],right=stops_times[['trip_id','departure_time','stop_id','arrival_time']], on='stop_id')
#print(df[(df['stop_name'].str.contains('Paris-Est', na=False)) & ((df['departure_time']<='10:00:00'))][['stop_name','trip_id','departure_time']].sort_values(by=['departure_time']))

Unify stop id format by keeping only the digits. This is necessary because the same station can have different character that are no interest for our purpose before the id. 

In [15]:
def formatID(id):
    return ''.join([i for i in id if i.isdigit()])

ustops['stop_id']=ustops['stop_id'].apply(formatID)

In [16]:
df_info = pd.DataFrame(ustops.dtypes).T.rename(index= {0: 'column type'})
df_info = df_info.append(pd.DataFrame(ustops.isnull().sum()).T.rename(index= {0: 'null values (nb)'}))
df_info = df_info.append(pd.DataFrame(ustops.isnull().sum() / ustops.shape[0] * 100).T.rename(index= {0: "null values (%)"}))

/tmp/ipykernel_6848/2615840537.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_info = df_info.append(pd.DataFrame(ustops.isnull().sum()).T.rename(index= {0: 'null values (nb)'}))
/tmp/ipykernel_6848/2615840537.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_info = df_info.append(pd.DataFrame(ustops.isnull().sum() / ustops.shape[0] * 100).T.rename(index= {0: "null values (%)"}))


In [18]:
df_info

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
column type,object,object,float64,float64,float64,float64,float64,int64,object
null values (nb),0,0,3842,0,0,3842,3842,0,3834
null values (%),0.0,0.0,100.0,0.0,0.0,100.0,100.0,0.0,99.791775


Remove columns that contains null values.

In [10]:
ustops=ustops.dropna(axis='columns')


ustops.to_csv('stops.csv')

To load csv file in neo4j
LOAD CSV WITH HEADERS FROM 'file:/stops.csv' AS line
 MERGE (s:Station {name: line.stop_name, id: line.stop_id, lat: line.stop_lat, lon:line.stop_lon, locationType:line.location_type})

not needed

# Create trips between station

In [6]:
stops_times['stop_id']=stops_times['stop_id'].apply(formatID)


In [7]:
stops_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,OCESN037071R0100119847,23:05:00,23:05:00,87381509,0,NaN,0,0,NaN
1,OCESN037071R0100119847,23:35:00,23:35:00,87415604,1,NaN,0,0,NaN
2,OCESN037071R0100119847,23:55:00,23:55:00,87415620,2,NaN,0,0,NaN
3,OCESN037071R0100119847,24:25:00,24:25:00,87415877,3,NaN,0,0,NaN
4,OCESN037071R0100119847,24:45:00,24:45:00,87411207,4,NaN,0,0,NaN
...,...,...,...,...,...,...,...,...,...
209124,OCEZW861292F0200254327,18:15:00,18:15:00,87543165,13,NaN,0,0,NaN
209125,OCESN082201R04004~NAV55729,08:45:00,08:45:00,87471003,0,NaN,0,0,NaN
209126,OCESN082201R04004~NAV55729,09:55:00,09:55:00,87471607,1,NaN,0,0,NaN
209127,OCESN082201R03003~NAV55728,08:40:00,08:40:00,87471003,0,NaN,0,0,NaN


In [8]:
rel=stops_times.groupby('trip_id',as_index=False)['arrival_time','departure_time','stop_id'].sum()

/tmp/ipykernel_11920/2245694226.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  rel=stops_times.groupby('trip_id',as_index=False)['arrival_time','departure_time','stop_id'].sum()


In [9]:
rel

,trip_id,arrival_time,departure_time,stop_id
0,OCESN003100F140147152,05:20:0005:35:0005:49:0006:12:0007:38:00,05:20:0005:36:0005:50:0006:15:0007:38:00,8741301387413344874133858741101787384008
1,OCESN003100F150157151,05:20:0005:35:0005:49:0006:12:0007:40:00,05:20:0005:36:0005:50:0006:15:0007:40:00,8741301387413344874133858741101787384008
2,OCESN003101F050057154,06:40:0008:02:0008:25:0008:39:0008:54:00,06:40:0008:05:0008:26:0008:40:0008:54:00,8738400887411017874133858741334487413013
3,OCESN003101F05005~NIB54832,06:40:0008:02:0008:25:0008:39:0008:54:00,06:40:0008:05:0008:26:0008:40:0008:54:00,8738400887411017874133858741334487413013
4,OCESN003101F060067153,06:38:0008:02:0008:25:0008:39:0008:54:00,06:38:0008:05:0008:26:0008:40:0008:54:00,8738400887411017874133858741334487413013
...,...,...,...,...
24106,OCEZW861293F0200254328,17:32:0017:42:0017:47:0017:55:0017:59:0018:04:...,17:32:0017:43:0017:48:0017:55:0017:59:0018:06:...,8754316587576082875761408757615787576165875761...
24107,OCEZW861295F0200254329,18:42:0018:52:0018:57:0019:05:0019:09:0019:14:...,18:42:0018:53:0018:58:0019:05:0019:09:0019:16:...,8754316587576082875761408757615787576165875761...
24108,OCEZW861296F0200254330,18:44:0018:51:0018:57:0019:04:0019:11:0019:13:...,18:44:0018:51:0018:58:0019:05:0019:11:0019:13:...,8757621587576231875762238757687687576199876992...
24109,OCEZW861297F0200254331,18:53:0019:03:0019:08:0019:16:0019:20:0019:25:...,18:53:0019:04:0019:09:0019:16:0019:20:0019:27:...,8754316587576082875761408757615787576165875761...


In [10]:
rel['arrival_time']=rel['arrival_time'].map(lambda x:[x[i:i+8]for i in range(8,len(x),8)])
rel['departure_time']=rel['departure_time'].map(lambda x:[x[i:i+8]for i in range(0,len(x)-8,8)])
rel['stop_id']=rel['stop_id'].map(lambda x:[x[i:i+8]for i in range(0,len(x),8)])

In [11]:
rel['stop_id']=rel['stop_id'].map(lambda x:[[x[i],x[i+1]]for i in range(len(x)-1)])

In [12]:
rel

,trip_id,arrival_time,departure_time,stop_id
0,OCESN003100F140147152,"[05:35:00, 05:49:00, 06:12:00, 07:38:00]","[05:20:00, 05:36:00, 05:50:00, 06:15:00]","[[87413013, 87413344], [87413344, 87413385], [..."
1,OCESN003100F150157151,"[05:35:00, 05:49:00, 06:12:00, 07:40:00]","[05:20:00, 05:36:00, 05:50:00, 06:15:00]","[[87413013, 87413344], [87413344, 87413385], [..."
2,OCESN003101F050057154,"[08:02:00, 08:25:00, 08:39:00, 08:54:00]","[06:40:00, 08:05:00, 08:26:00, 08:40:00]","[[87384008, 87411017], [87411017, 87413385], [..."
3,OCESN003101F05005~NIB54832,"[08:02:00, 08:25:00, 08:39:00, 08:54:00]","[06:40:00, 08:05:00, 08:26:00, 08:40:00]","[[87384008, 87411017], [87411017, 87413385], [..."
4,OCESN003101F060067153,"[08:02:00, 08:25:00, 08:39:00, 08:54:00]","[06:38:00, 08:05:00, 08:26:00, 08:40:00]","[[87384008, 87411017], [87411017, 87413385], [..."
...,...,...,...,...
24106,OCEZW861293F0200254328,"[17:42:00, 17:47:00, 17:55:00, 17:59:00, 18:04...","[17:32:00, 17:43:00, 17:48:00, 17:55:00, 17:59...","[[87543165, 87576082], [87576082, 87576140], [..."
24107,OCEZW861295F0200254329,"[18:52:00, 18:57:00, 19:05:00, 19:09:00, 19:14...","[18:42:00, 18:53:00, 18:58:00, 19:05:00, 19:09...","[[87543165, 87576082], [87576082, 87576140], [..."
24108,OCEZW861296F0200254330,"[18:51:00, 18:57:00, 19:04:00, 19:11:00, 19:13...","[18:44:00, 18:51:00, 18:58:00, 19:05:00, 19:11...","[[87576215, 87576231], [87576231, 87576223], [..."
24109,OCEZW861297F0200254331,"[19:03:00, 19:08:00, 19:16:00, 19:20:00, 19:25...","[18:53:00, 19:04:00, 19:09:00, 19:16:00, 19:20...","[[87543165, 87576082], [87576082, 87576140], [..."


In [13]:
rel['trip_id']=rel.apply(lambda x : [x['trip_id']for i in range(len(x['stop_id']))],axis=1)

In [14]:
rel['start']=rel.apply(lambda x : [x['stop_id'][i][0]for i in range(len(x['stop_id']))],axis=1)
rel['dest']=rel.apply(lambda x : [x['stop_id'][i][1]for i in range(len(x['stop_id']))],axis=1)

In [15]:
rel

,trip_id,arrival_time,departure_time,stop_id,start,dest
0,"[OCESN003100F140147152, OCESN003100F140147152,...","[05:35:00, 05:49:00, 06:12:00, 07:38:00]","[05:20:00, 05:36:00, 05:50:00, 06:15:00]","[[87413013, 87413344], [87413344, 87413385], [...","[87413013, 87413344, 87413385, 87411017]","[87413344, 87413385, 87411017, 87384008]"
1,"[OCESN003100F150157151, OCESN003100F150157151,...","[05:35:00, 05:49:00, 06:12:00, 07:40:00]","[05:20:00, 05:36:00, 05:50:00, 06:15:00]","[[87413013, 87413344], [87413344, 87413385], [...","[87413013, 87413344, 87413385, 87411017]","[87413344, 87413385, 87411017, 87384008]"
2,"[OCESN003101F050057154, OCESN003101F050057154,...","[08:02:00, 08:25:00, 08:39:00, 08:54:00]","[06:40:00, 08:05:00, 08:26:00, 08:40:00]","[[87384008, 87411017], [87411017, 87413385], [...","[87384008, 87411017, 87413385, 87413344]","[87411017, 87413385, 87413344, 87413013]"
3,"[OCESN003101F05005~NIB54832, OCESN003101F05005...","[08:02:00, 08:25:00, 08:39:00, 08:54:00]","[06:40:00, 08:05:00, 08:26:00, 08:40:00]","[[87384008, 87411017], [87411017, 87413385], [...","[87384008, 87411017, 87413385, 87413344]","[87411017, 87413385, 87413344, 87413013]"
4,"[OCESN003101F060067153, OCESN003101F060067153,...","[08:02:00, 08:25:00, 08:39:00, 08:54:00]","[06:38:00, 08:05:00, 08:26:00, 08:40:00]","[[87384008, 87411017], [87411017, 87413385], [...","[87384008, 87411017, 87413385, 87413344]","[87411017, 87413385, 87413344, 87413013]"
...,...,...,...,...,...,...
24106,"[OCEZW861293F0200254328, OCEZW861293F020025432...","[17:42:00, 17:47:00, 17:55:00, 17:59:00, 18:04...","[17:32:00, 17:43:00, 17:48:00, 17:55:00, 17:59...","[[87543165, 87576082], [87576082, 87576140], [...","[87543165, 87576082, 87576140, 87576157, 87576...","[87576082, 87576140, 87576157, 87576165, 87576..."
24107,"[OCEZW861295F0200254329, OCEZW861295F020025432...","[18:52:00, 18:57:00, 19:05:00, 19:09:00, 19:14...","[18:42:00, 18:53:00, 18:58:00, 19:05:00, 19:09...","[[87543165, 87576082], [87576082, 87576140], [...","[87543165, 87576082, 87576140, 87576157, 87576...","[87576082, 87576140, 87576157, 87576165, 87576..."
24108,"[OCEZW861296F0200254330, OCEZW861296F020025433...","[18:51:00, 18:57:00, 19:04:00, 19:11:00, 19:13...","[18:44:00, 18:51:00, 18:58:00, 19:05:00, 19:11...","[[87576215, 87576231], [87576231, 87576223], [...","[87576215, 87576231, 87576223, 87576876, 87576...","[87576231, 87576223, 87576876, 87576199, 87699..."
24109,"[OCEZW861297F0200254331, OCEZW861297F020025433...","[19:03:00, 19:08:00, 19:16:00, 19:20:00, 19:25...","[18:53:00, 19:04:00, 19:09:00, 19:16:00, 19:20...","[[87543165, 87576082], [87576082, 87576140], [...","[87543165, 87576082, 87576140, 87576157, 87576...","[87576082, 87576140, 87576157, 87576165, 87576..."


In [16]:
df=pd.DataFrame({'start':rel['start'].sum(),'dest':rel['dest'].sum(),'trip_id':rel['trip_id'].sum(),'departure_time':rel['departure_time'].sum(),'arrival_time':rel['arrival_time'].sum()})

In [26]:
df[df.duplicated(subset="start")]

,start,dest,trip_id,departure_time,arrival_time
4,87413013,87413344,OCESN003100F150157151,05:20:00,05:35:00
5,87413344,87413385,OCESN003100F150157151,05:36:00,05:49:00
6,87413385,87411017,OCESN003100F150157151,05:50:00,06:12:00
7,87411017,87384008,OCESN003100F150157151,06:15:00,07:40:00
9,87411017,87413385,OCESN003101F050057154,08:05:00,08:25:00
...,...,...,...,...,...
185013,87576173,87576165,OCEZW861298F0200254332,19:37:00,19:42:00
185014,87576165,87576157,OCEZW861298F0200254332,19:42:00,19:46:00
185015,87576157,87576140,OCEZW861298F0200254332,19:46:00,19:53:00
185016,87576140,87576082,OCEZW861298F0200254332,19:54:00,19:58:00


In [18]:
df.to_csv('rel.csv')

### Load data in neo4j

Put rel.csv in the import directory of your dbms. (on ubuntu "~/.config/'Neo4j Desktop'/Application/relate-data/dbmss/dbms-<dbmss id>/import")

Open neo4j browser then type
```
LOAD CSV WITH HEADERS FROM "file:///rel.csv" AS row
MATCH (s1:Station {id: row.start})
MATCH (s2:Station {id: row.dest})
CALL apoc.create.relationship(s1, row.trip_id, {departure_time:row.departure_time,arrival_time:row.arrival_time}, s2) yield rel return rel
```
```
match (n:Station)-[r]-(:Station) with r as r, split(r.departure_time,":") as dep,split(r.arrival_time,":") as ar set r.duration=(toFloat(ar[0])-toFloat(dep[0]))*60+toFloat(ar[1])-toFloat(dep[1])
```


# Format service to load it neo4j


calendar_dates are the exceptions in the calendar where the service is unavailable. The meaning of exception type stays unknown.

In [48]:
calendar_dates

,service_id,date,exception_type
0,9939,20200413,2
1,8655,20200418,1
2,8655,20200516,1
3,12563,20200425,1
4,12563,20200508,1
...,...,...,...
38155,5931,20200501,2
38156,5931,20200508,2
38157,12834,20200413,2
38158,12834,20200501,2


In [49]:
calendar_dates["date"]=calendar_dates["date"].astype(str)

Groups all the exceptions by service id.

In [50]:
calendar_dates=calendar_dates.groupby("service_id",as_index=False)["date"].sum()

In [51]:
calendar_dates['date']=calendar_dates['date'].map(lambda x:','.join([x[i:i+8]for i in range(0,len(x),8)]))

In [52]:
calendar_dates['date'].min()

'20200220'

#### **calendar** represents the period of availability of the service and the days on which it is available.

In [53]:
calendar

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,9939,1,1,1,1,0,0,0,20200220,20200520
1,8655,0,0,0,0,0,1,0,20200314,20200405
2,2341,0,0,0,0,0,1,0,20200222,20200307
3,12564,0,0,0,0,0,0,1,20200419,20200426
4,12565,0,0,0,0,0,1,0,20200314,20200404
...,...,...,...,...,...,...,...,...,...,...
5010,11448,1,1,1,1,1,1,0,20200222,20200520
5011,11283,1,1,1,1,1,0,0,20200224,20200520
5012,5931,1,1,1,1,1,0,0,20200220,20200520
5013,12834,1,1,1,1,1,0,0,20200220,20200520


Create a list of days from the dataframe.

In [54]:
calendar["days"]=calendar.apply(lambda x : ','.join([i for i in ["monday","tuesday","wednesday","thursday","friday","saturday","sunday"] if x[i]==1]),axis=1)

In [55]:
calendar.loc[calendar['days']==','.join(["monday","tuesday","wednesday","thursday","friday","saturday","sunday"])]

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,days
111,12631,1,1,1,1,1,1,1,20200316,20200405,"monday,tuesday,wednesday,thursday,friday,satur..."
124,12646,1,1,1,1,1,1,1,20200309,20200405,"monday,tuesday,wednesday,thursday,friday,satur..."
140,11414,1,1,1,1,1,1,1,20200316,20200405,"monday,tuesday,wednesday,thursday,friday,satur..."
146,2424,1,1,1,1,1,1,1,20200220,20200307,"monday,tuesday,wednesday,thursday,friday,satur..."
147,2425,1,1,1,1,1,1,1,20200308,20200405,"monday,tuesday,wednesday,thursday,friday,satur..."
...,...,...,...,...,...,...,...,...,...,...,...
4962,12154,1,1,1,1,1,1,1,20200220,20200520,"monday,tuesday,wednesday,thursday,friday,satur..."
4965,4557,1,1,1,1,1,1,1,20200220,20200520,"monday,tuesday,wednesday,thursday,friday,satur..."
4976,4160,1,1,1,1,1,1,1,20200220,20200520,"monday,tuesday,wednesday,thursday,friday,satur..."
4981,10129,1,1,1,1,1,1,1,20200220,20200520,"monday,tuesday,wednesday,thursday,friday,satur..."


In [56]:
calendar

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,days
0,9939,1,1,1,1,0,0,0,20200220,20200520,"monday,tuesday,wednesday,thursday"
1,8655,0,0,0,0,0,1,0,20200314,20200405,saturday
2,2341,0,0,0,0,0,1,0,20200222,20200307,saturday
3,12564,0,0,0,0,0,0,1,20200419,20200426,sunday
4,12565,0,0,0,0,0,1,0,20200314,20200404,saturday
...,...,...,...,...,...,...,...,...,...,...,...
5010,11448,1,1,1,1,1,1,0,20200222,20200520,"monday,tuesday,wednesday,thursday,friday,saturday"
5011,11283,1,1,1,1,1,0,0,20200224,20200520,"monday,tuesday,wednesday,thursday,friday"
5012,5931,1,1,1,1,1,0,0,20200220,20200520,"monday,tuesday,wednesday,thursday,friday"
5013,12834,1,1,1,1,1,0,0,20200220,20200520,"monday,tuesday,wednesday,thursday,friday"


Merge calendar and calendar_dates in order to have all the informations about the services in one dataframe. 

In [57]:
df_merged = calendar.merge(calendar_dates, on='service_id', how='outer')

# Concatenate the other columns when there are multiple rows with the same ID
#df_merged = df_merged.groupby('service_id').agg(lambda x: ','.join(x))

df_merged

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,days,date
0,9939,1.0,1.0,1.0,1.0,0.0,0.0,0.0,20200220.0,20200520.0,"monday,tuesday,wednesday,thursday",20200413
1,8655,0.0,0.0,0.0,0.0,0.0,1.0,0.0,20200314.0,20200405.0,saturday,"20200418,20200516"
2,2341,0.0,0.0,0.0,0.0,0.0,1.0,0.0,20200222.0,20200307.0,saturday,NaN
3,12564,0.0,0.0,0.0,0.0,0.0,0.0,1.0,20200419.0,20200426.0,sunday,NaN
4,12565,0.0,0.0,0.0,0.0,0.0,1.0,0.0,20200314.0,20200404.0,saturday,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5618,12817,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200426
5619,12818,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200413
5620,12823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200508
5621,12828,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200508


In [60]:
calendar_dates.loc[calendar_dates['service_id']==12818]

,service_id,date
4268,12818,20200413


In [59]:
len(trips['service_id'].unique())

5623

Some service_id are in calendar but not in calendar_date and some service_id are in calendar_date but not in calendar.

So we will replace nan value in days by all the day of the week ["monday","tuesday","wednesday","thursday","friday","saturday","sunday"]
The value in date will be replaced by an empy array

In [ ]:
df_merged["days"]=df_merged["days"].fillna(",".join(["monday","tuesday","wednesday","thursday","friday","saturday","sunday"]))

The value in date will be replaced by an empy array

In [ ]:
df_merged['date']=df_merged["date"].fillna("none")

The nan start dates will be replaced by 20200220 which is the first start date.
The nan end dates will be replaced by 20200520 which is the last end date.


In [ ]:
df_merged['start_date']=df_merged['start_date'].fillna(20200220)
df_merged['end_date']=df_merged['end_date'].fillna(20200520)

In [ ]:
df_merged

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,days,date
0,9939,1.0,1.0,1.0,1.0,0.0,0.0,0.0,20200220.0,20200520.0,"monday,tuesday,wednesday,thursday",20200413
1,8655,0.0,0.0,0.0,0.0,0.0,1.0,0.0,20200314.0,20200405.0,saturday,"20200418,20200516"
2,2341,0.0,0.0,0.0,0.0,0.0,1.0,0.0,20200222.0,20200307.0,saturday,none
3,12564,0.0,0.0,0.0,0.0,0.0,0.0,1.0,20200419.0,20200426.0,sunday,none
4,12565,0.0,0.0,0.0,0.0,0.0,1.0,0.0,20200314.0,20200404.0,saturday,none
...,...,...,...,...,...,...,...,...,...,...,...,...
5618,12817,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200220.0,20200520.0,"monday,tuesday,wednesday,thursday,friday,satur...",20200426
5619,12818,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200220.0,20200520.0,"monday,tuesday,wednesday,thursday,friday,satur...",20200413
5620,12823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200220.0,20200520.0,"monday,tuesday,wednesday,thursday,friday,satur...",20200508
5621,12828,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200220.0,20200520.0,"monday,tuesday,wednesday,thursday,friday,satur...",20200508


In [ ]:
df_merged.to_csv('service.csv')

```
LOAD CSV WITH HEADERS FROM 'file:/service.csv' AS line
Merge (s:Service {id: line.service_id, unavailability:split(line.date,','), start_date:line.start_date,end_date:line.end_date}) 
with s, split(line.days,',') as days
UNWIND days as day 
MERGE (d:Day {name:day})
MERGE (d)-[r:AVAILABLE]-(s)
```

# Load trips into neo4j
We need to be able to get a list of trip ids that are available on a particular day in order to create the gds graph to run the astar algorithm.

In [ ]:
trips

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
0,OCE1506035,9939,OCESN037071R0100119847,37071,1,NaN,NaN
1,OCE1506035,8655,OCESN003191F020027314,3191,1,NaN,NaN
2,OCE1506035,12563,OCESN003191F06006~NIR54906,3191,1,NaN,NaN
3,OCE1506035,2341,OCESN003191F010017311,3191,1,NaN,NaN
4,OCE1506035,12564,OCESN003117F14014~NIR54870,3117,1,NaN,NaN
...,...,...,...,...,...,...,...
24106,OCEZW-87543165-87576215,11256,OCEZW861288F0300354322,861288,0,NaN,NaN
24107,OCEZW-87543165-87576215,513,OCEZW861296F0200254330,861296,0,NaN,NaN
24108,OCEZW-87543165-87576215,540,OCEZW861292F0200254327,861292,0,NaN,NaN
24109,OCESN-87471607-87471003,12834,OCESN082201R04004~NAV55729,82201,1,NaN,NaN




```
LOAD CSV WITH HEADERS FROM 'file:/trips.txt' AS line
Match (s:Service {id:line.service_id})
Merge (t:Trip {id: line.trip_id, route_id:line.route_id, headsign:line.trip_headsign, direction:line.direction_id})-[:BELONGS_TO]-(s)
```

Now  ```Match(d:Day {name: "monday"})-[]-(s:Service)-[]-(t:Trip) where s.start_date<"20200309"<s.end_date and not "20200309" in s.unavailability return t.id``` returns the trip id that are available the 3rd of march 2020

In [ ]:
datetime.datetime(2020, 3, 9).weekday()

0

In [ ]:
str(["fdf","fdd","fefd"])

"['fdf', 'fdd', 'fefd']"